# UIExtraction Part1: Extract

In [ ]:
"""
Autoencoder
Model compile and train.

Licensed under the GPL-3.0 License (see LICENSE for details)
Written by NPU-Franklin@Github.com
"""

Designed to extract the most valuable features from images using autoencoder(AE).

## 1. Importing

In [ ]:
import os
import cv2

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mping

from scipy import *
from tensorflow import keras
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, AveragePooling2D, UpSampling2D
from keras.callbacks import TensorBoard
from tensorflow.python.keras.utils import plot_model

In [ ]:
print("keras_version: "+keras.__version__)
print("Tensorflow_version: "+tf.__version__)

## 2. Configuration

### Set font size, figure size and the gpu device CUDA using.

In [ ]:
plt.rcParams['font.size'] = 18
plt.rcParams['figure.figsize'] = (18, 24)

os.environ['CUDA_VISIBLE_DEVICES'] = '/gpu:0'

## 3. Load and preprocess images

### Define basic information of images.

In [ ]:
img_width, img_height, channels = 1536, 2048, 3
input_shape = (img_height, img_width, channels)

## Define image resize funtion.

In [ ]:
def resize_data(dir):
    files = ["%s/%s" % (dir, num) for num in os.listdir(dir)]
    for file in files:
        img_old = cv2.imread(file, cv2.IMREAD_UNCHANGED)
        img_new = cv2.resize(img_old, (1536, 2048), interpolation=cv2.INTER_CUBIC)
        save_path = dir+'_resize/'
        img_name = os.path.basename(file)
        if os.path.exists(save_path):
            print(file)
            try:
                save_img = save_path + img_name
                cv2.imwrite(save_img, img_new, [int(cv2.IMWRITE_PNG_COMPRESSION),1])
                print('succeed!')
            except:
                print('False!')
        else:
            os.mkdir(save_path)
            print(file)
            try:
                save_img = save_path + img_name
                cv2.imwrite(save_img, img_new, [int(cv2.IMWRITE_PNG_COMPRESSION),1])
                print('succeed!')
            except:
                print('False!')

### Define load_data function.

In [ ]:
def load_data(dir):
    if not os.path.exists(dir):
        dir = "." + dir
    resize_data(dir)
    dir = dir + '_resize'
    files = ["%s/%s" % (dir, num) for num in os.listdir(dir)]
    arr = np.empty((len(files), img_height, img_width, channels), dtype=np.float64)
    for i, imgfile in enumerate(files):
        img = load_img(imgfile)
        x = img_to_array(img).reshape(img_height, img_width, channels)
        x = x.astype('float64')/255
        arr[i] = x
    return arr

### Load data.

In [ ]:
train_data = load_data(r'./datasets/304x304')
print(train_data.shape)

### Plot an image of the dataset.

In [ ]:
plt.imshow(train_data[0])
plt.axis('off')
plt.show()

### Save the data as numpy format for future using.

In [ ]:
if os.path.exists(r"./data_numpyarray"):
    np.save('./data_numpyarray/dataset_304xx304_reshaped.npy', train_data)
else:
    np.save('../data_numpyarray/dataset_304xx304_reshaped.npy', train_data)

## 4. Build and compile the model

In [ ]:
model = Sequential()

### Encoder part

In [ ]:
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape, padding='same'))
model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
model.add(Conv2D(8, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
model.add(Conv2D(4, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))

### Decoder part

In [ ]:
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(4, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(8, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(Conv2D(3, kernel_size=(3, 3), activation='sigmoid', padding='same'))

### Compile

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Summary

In [ ]:
model.summary()

## 5. Training

In [ ]:
model.fit(train_data, train_data, epochs=100, shuffle=True)

## 6. Save the model

In [ ]:
if os.path.exists(r"./results"):
    model.save('./results/model/CAE_304x304_reshaped.h5')
else:
    model.save('../results/model/CAE_304x304_reshaped.h5')

## 7. Checkout the result.

In [ ]:
img_decoded = model.predict(train_data[:1])
plt.figure()
plt.subplot(1, 2, 1)
plt.axis('off')
plt.title("Before encoding")
plt.imshow(train_data[:1].reshape(img_height, img_width, channels))
plt.subplot(1, 2, 2)
plt.axis('off')
plt.title("After encoding")
plt.imshow(img_decoded.reshape(img_height, img_width, channels))